# Implement a physically correct distribution

In order ot

In [2]:
import numpy
import pandas
import generate_structure

from IPython.display import Image
from IPython.core.display import HTML 

In [6]:
Image(url= "./z.png",  width=500)